# Electric Fixed Wing sUAS Design

### Introduction
The following will overview the design process for an electric fixed wing small unmanned aerial system (sUAS) as well as providing code for designing one. Due to the nature of the vehicle being designed, the operational conditions are assumed to be standard sea level. All units are SI unless otherwise noted.

In [11]:
# Setup

# Imports
import numpy as np
import matplotlib.pyplot as plt

# Constants
pi = 3.1415926

# Physical Constants
gravity = 9.80665               # m/s^2

# Standard Sea Level Atmosphere
density = 1.225                 # kg/m^3
temp_C = 15                     # C
temp_K = 288.2                  # K
pressure = 101325               # Pa
speed_of_sound = 340.3          # m/s
dynamic_viscosity = 1.789e-5    # kg/m/s
kinematic_viscosity = 1.460e-5  # m^2/s
gas_constant = 287.1            # J/kg/K
temp_lapse_rate = 0.0065        # K/m
molar_mass = 0.0289652          # kg/mol
ratio_of_specific_heats = 1.4   # unitless

### Requirements

Any aircraft design process begins by defining requirements. These will likely be adjusted in an iterative process as certain combinations of requirements are mutually exclusive given a certain budget or current available technology.

In [12]:
# Requirements

W_payload = 0.5 * gravity  # N, required payload weight
range = 20000  # m, required range


### Initial Sizing Estimate

The first step of aircraft design is an estimate of gross takeoff weight given by the following formula

\begin{gathered}
W_0 = \frac{W_p}{1-\frac{W_b}{W_0}-\frac{W_e}{W_0}} 
\end{gathered}

Where

\begin{gathered}
W_0 = \text{Gross Takeoff Weight} \\
W_p = \text{Payload Weight (All Non-Flight-Critical Mass)} \\
\frac{W_b}{W_0} = \text{Battery Weight Fraction} \\
\frac{W_e}{W_0} = \text{Empty Weight Fraction} \\
\end{gathered}

Payload weight is a requirement. Battery Weight Fraction is estimated based on the mission requirements. Empty weight fraction is estimated based on the Gross Takeoff Weight. However, since Gross Takeoff Weight is what we are solving for, Empty Weight Fraction and Gross Takeoff Weight must be estimated iteratively.

Source: Raymer 6th Ed. Chapter 3

##### Battery Weight Fraction Estimate

Note that unlike standard fuel-based aircraft, electric aircraft use a constant-weight battery. The following method of estimating Battery Weight Fraction is used.

For a low altitude electric sUAS, I will assume climb and landing make up a relatively small portion of flight time and can therefore be treated identically to cruise. Therefore the Battery Weight Fraction can be estimated using only the Range, Estimated L/D and Estimated Powertrain Efficiency

\begin{gathered}
\frac{W_b}{W_0} = \frac{g*R}{3600*e*\frac{L}{D}*n}
\end{gathered}

where

\begin{gathered}
g = \text{gravity (m/s/s)} \\
R = \text{Range (m)} \\
e = \text {Battery Specific Energy (W*hr/kg)} \\
\frac{L}{D} = \text{Lift to Drag Ratio} \\
\eta = \text{Efficiency } (\eta_{propeller} * \eta_{motor} * \eta_{esc} * \eta_{battery} * \eta_{wire})
\end{gathered}


Source: Raymer 6th Ed. Chapter 20 Pg 759

Range is a Requirement and therefore does not require estimation.

Propeller efficiency is a function of advance ratio, but for initial sizing purposes we will assume it falls between 0.5 and 0.7.

See: https://m-selig.ae.illinois.edu/props/propDB.html

Wire efficiency is extremely high at such short lengths and is assumed to be 0.99. All other efficiencies are fairly high and have been roughly estimated as 0.9.

Specific Energy is estimated to be approximately 150 W*Hr/kg for RC Hobby-Grade LiPo Batteries. This can be higher or lower depending on battery quality.

See: https://www.rcgroups.com/forums/showthread.php?1767093-Battery-Load-Test-Comparisons

L/D Can be Estimated based on historical data from similar airplanes. An electric sUAS will likely have and L/D in the 5-15 range. For a physical intuition, L/D is equivalent to the unpowered glide slope of an airplane.

A method of L/D estimation is to use the formula

\begin{gathered}
\frac{L}{D}_{max} = K_{LD} * \sqrt{A_{wet}} \\

\text{where } K_{LD} = 9 \text{ for non-retractable gear, propeller aircraft} \\
A_{wet} = \frac{b^2}{S_{wet}} = \frac{A}{S_{wet}/S_{ref}} = \text{Wetted Aspect Ratio} \\
A = \frac{b^2}{S_{ref}} = \text{Aspect Ratio} \\
b = \text{Wing Span} \\
S_{wet} = \text{Wetted Area (Total Surface Area of the Airplane)} \\
S_{ref} = \text{Wing Area (Planform Area)} \\
\end{gathered}

Source: Raymer 6th Ed. Chapter 3


Apspect Ratio is selected as part of the initial rough configuration. A traditional general aviation propeller-recip aircraft will have an aspect ratio between 5-8. For the purposed of an electric sUAS, I will assume


Wetted Area Ratio, S<sub>wet</sub>/S<sub>ref</sub>, can be estimated by "eyeballing" the configuration compared to historical data 



See: https://www.atmos.albany.edu/daes/atmclasses/env327/References/UAS-jtech-d-13-00236%252E1.pdf

In [20]:
# Inputs
efficiency_propeller = .6  # propeller efficiency
efficiency_wire = .99  # wire effiency
efficiency_motor = .9  # motor efficiency
efficiency_esc = .9  # esc efficiency
efficiency_batt = .9  # battery efficiency

batt_specific_energy = 150  # W*Hr/kg, RC Grade Lipo Specific Energy

#wingspan_estimate = 1  # m, initial estimate for wingspan
aspect_ratio_estimate = 6  # initial estimate for aspect ratio
wetted_area_ratio_estimate = 4  # initial estimate for wetted area ratio
k_LD = 9

# Outputs
print(f"Required Range: {range} m")

efficiency_total = efficiency_propeller * efficiency_motor * efficiency_esc * efficiency_batt * efficiency_wire
print(f"Total Efficiency: {efficiency_total:.3f}")

print(f"Battery Specific Energy: {batt_specific_energy} W*hr/kg")

L_D_Max = k_LD * np.sqrt(aspect_ratio_estimate/wetted_area_ratio_estimate)
print(f"Max L/D: {L_D_Max:.1f}")

BMF = gravity * range / (3600 * batt_specific_energy * L_D_Max * efficiency_total)
print(f"Battery Mass Fraction: {BMF:.3f}")


Required Range: 20000 m
Total Efficiency: 0.433
Battery Specific Energy: 150 W*hr/kg
Max L/D: 11.0
Battery Mass Fraction: 0.076


##### Empty Weight Fraction Estimate